<a href="https://colab.research.google.com/github/zzwony/Start_0920/blob/main/01_11_bert_pair_cls_deplot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 의존성 패키지 설치

In [ ]:
!pip install ratsnlp

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언한다.

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationDeployArguments
args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="/gdrive/MyDrive/nlpbook/checkpoint-paircls1",
    max_seq_length=64,
)

downstream_model_checkpoint_fpath: /gdrive/MyDrive/nlpbook/checkpoint-paircls1/epoch=0-val_loss=0.86.ckpt


# 모델 로딩
파인튜닝을 마친 모델과 토크나이저를 읽어 들인다.

In [ ]:
import torch
from transformers import BertConfig, BertForSequenceClassification

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu')
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels = fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),## shape.numel()은 텐서의 갯수를 나타냄
)

model = BertForSequenceClassification(pretrained_model_config)##  초기화된 모델
model.load_state_dict({k.replace('model.', ''): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})## 학습시킨다.
model.eval()

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
# 토크나이저 초기화
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

# 인퍼런스 함수 선언
인퍼런스 함수를 선언한다.

In [ ]:
def inference_fn(premise, hypothesis):## premise는 전제, hypothesis는 가설
    inputs = tokenizer(
        [(premise, hypothesis)],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,## 너무 길면 자르는거 허용한다.
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})## torch.tensor(v)는 밸류값은 텐서로 바꿔줘야한다.
        prob = outputs.logits.softmax(dim=1)
        entailment_prob = round(prob[0][0].item(), 2)
        contradiction_prob = round(prob[0][1].item(), 2)
        neutral_prob = round(prob[0][2].item(), 2)
        if torch.argmax(prob) == 0:
            pred = "참 (entailment)"
        elif torch.argmax(prob) == 1:
            pred = "거짓 (contradiction)"
        else:
            pred = "중립 (neutral)"
    return {
        'premise': premise,
        'hypothesis': hypothesis,
        'prediction': pred,
        'entailment_data': f"참 {entailment_prob}",
        'contradiction_data': f"거짓 {contradiction_prob}",
        'neutral_data': f"중립 {neutral_prob}",
        'entailment_width': f"{entailment_prob * 100}%",
        'contradiction_width': f"{contradiction_prob * 100}%",
        'neutral_width': f"{neutral_prob * 100}%",
    }

In [ ]:
벨류값은 왜 텐서로 바꿔줘야하지?

In [ ]:
premise = '오늘은 금요일이다.'
hypothesis = '내일은 주말이다.'
inference_fn(premise, hypothesis)

{'premise': '오늘은 금요일이다.',
 'hypothesis': '내일은 주말이다.',
 'prediction': '참 (entailment)',
 'entailment_data': '참 0.77',
 'contradiction_data': '거짓 0.16',
 'neutral_data': '중립 0.07',
 'entailment_width': '77.0%',
 'contradiction_width': '16.0%',
 'neutral_width': '7.000000000000001%'}

# 웹서비스 만들기 준비



In [ ]:
# !rm -rf /root/.ngrok2

In [ ]:
# !mkdir /root/.ngrok2 && echo 'authtoken: 2K7I7O1DpP2jRhLZkbgOyok0X8H_6ZVLBrT25o6NA81BJMC9a' > /root/.ngrok2/ngrok.yml

mkdir: cannot create directory ‘/root/.ngrok2’: File exists


In [ ]:
from ratsnlp.nlpbook.classification import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

 * Serving Flask app "ratsnlp.nlpbook.classification.deploy" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6e29-35-188-153-255.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
